In [1]:
# Import necessary libraries
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration
import faiss
import numpy as np

In [2]:
# Step 2: Extract Text (You can manually extract text from PDF)
text_chapter1 = "Text from chapter 1"
text_chapter2 = "Text from chapter 2"

In [3]:
# Step 3: Indexing using Faiss
index = faiss.IndexFlatIP(768)  # Assuming embeddings of size 768
retriever = RagRetriever(
    RagTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base"),
    RagTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base"),
)
embeddings_chapter1 = np.random.rand(10, 768).astype(np.float32)  # Example embeddings for chapter 1
embeddings_chapter2 = np.random.rand(10, 768).astype(np.float32)  # Example embeddings for chapter 2
index.add(embeddings_chapter1)
index.add(embeddings_chapter2)

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

C:\Users\U6054350\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\U6054350\.cache\huggingface\hub\models--facebook--dpr-question_encoder-single-nq-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You are using a model of type dpr to instantiate a model of type rag. 

AssertionError: Config has to be initialized with question_encoder and generator config

In [ ]:
# Step 4: Load Question Answering Model
qa_model = RagTokenForGeneration.from_pretrained("facebook/bart-large")

In [ ]:
# Step 5: RAG Pipeline
def rag_pipeline(question):
    # Retrieve
    retriever_output = retriever.retrieve(question)
    passage_ids = [doc["doc_id"] for doc in retriever_output]
    passages = [text_chapter1, text_chapter2]  # Actual passages corresponding to passage_ids
    passages_embeddings = np.concatenate([embeddings_chapter1, embeddings_chapter2])

    # Generate
    qa_inputs = qa_model.retriever_tokenizer(
        question,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=32,
    )
    qa_outputs = qa_model.generate(
        **qa_inputs,
        retriever=passages,
        embeddings=passages_embeddings,
        num_beams=4,
        num_return_sequences=1,
    )
    
    # Decode and return answer
    answer = qa_model.retriever_tokenizer.decode(qa_outputs[0], skip_special_tokens=True)
    return answer

In [ ]:
# Example usage
question = "What is biology?"
answer = rag_pipeline(question)
print("Answer:", answer)